In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project-X-Lip-Reading/Model Architecture

/content/drive/MyDrive/Project-X-Lip-Reading/Model Architecture


In [ ]:
import pickle
import time
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, MaxPooling3D
from tensorflow.keras.layers import Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

In [ ]:
with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/train/X_train.pkl', 'rb') as f:
   X_train = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/train/y_train.pkl', 'rb') as f:
   y_train = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/test/X_test.pkl', 'rb') as f:
   X_test = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/test/y_test.pkl', 'rb') as f:
   y_test = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/val/X_val.pkl', 'rb') as f:
   X_val = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/val/y_val.pkl', 'rb') as f:
   y_val = pickle.load(f)

In [ ]:
def build_3d_cnn_model(input = (22, 100, 100, 1), num_classes = 10, learning_rate = 0.01, dropout_rate = 0.5):

    model = Sequential()

    # 1st Layer group
    model.add(Conv3D(32, (3, 3, 3), strides = 1, input_shape=input, activation='relu', padding='valid'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

    # 2nd Layer group
    model.add(Conv3D(64, (3, 3, 3), activation='relu', strides=1))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

    # 2nd Layer group
    model.add(Conv3D(128, (3, 3, 3), activation='relu', strides=1))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

    # Reshape
    model.add(Reshape((128, 1*10*10)))

    # LSTMS Layer
    model.add(LSTM(32, return_sequences=True))
    model.add(Dropout(dropout_rate))

    # Flatten Layer
    model.add((Flatten()))

    # Dense & Dropout Layers
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Softmax Layer
    model.add(Dense(num_classes, activation='softmax'))

    # Model Compilation
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'dropout_rate': [0.2, 0.3, 0.4, 0.5],
}

In [ ]:
# Build Model
model = KerasClassifier(build_fn=build_3d_cnn_model, epochs=30, learning_rate = 0.01, batch_size=32, dropout_rate = 0.2)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=3, verbose=0)

t1 = time.time()
random_search.fit(X_train, y_train)
t2 = time.time()

print("Best hyperparameters:",random_search.best_params_)
print("Best hyperparameters:",random_search.best_score_)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequ

Best hyperparameters: {'learning_rate': 0.001, 'dropout_rate': 0.2, 'batch_size': 16}
Best hyperparameters: 0.5119047619047619
